In [1]:
pip install git+https://github.com/Open-Reaction-Database/ord-schema.git

  Cloning https://github.com/Open-Reaction-Database/ord-schema.git to /private/var/folders/g5/gfzgn6710bl4fnkqby736jyh0000gn/T/pip-req-build-paoejibf
  Running command git clone --filter=blob:none --quiet https://github.com/Open-Reaction-Database/ord-schema.git /private/var/folders/g5/gfzgn6710bl4fnkqby736jyh0000gn/T/pip-req-build-paoejibf
  Resolved https://github.com/Open-Reaction-Database/ord-schema.git to commit b2b10d44a3a631165d376386d1c385336c64e733
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ord_schema.message_helpers import load_message, write_message
from ord_schema.proto import dataset_pb2, reaction_pb2
import pandas as pd
import csv

In [3]:
# load the binary ord file
dataset = load_message("buchwald.pb.gz", dataset_pb2.Dataset)

In [4]:
# save the ord file as human readable text
write_message(dataset, "buchwald_hartwig_database.pbtxt")

In [11]:
from ord_schema.proto import reaction_pb2  # à adapter si nécessaire

def get_identifier_value(identifiers, id_type_code):
    for identifier in identifiers:
        if identifier.type == id_type_code:
            return identifier.value
    return ""

def get_smiles_from_components(components):
    smiles_list = []
    for comp in components:
        for iden in comp.identifiers:
            if iden.type == "SMILES":
                smiles_list.append(iden.value)
    return ";".join(smiles_list) if smiles_list else ""

with open("test3.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "reaction_id",
        "reaction_index_CUSTOM",
        "reaction_type",
        "inputs_metal_and_ligand_SMILES",
        "inputs_aryl_halide_SMILES",
        "inputs_amine_SMILES",
        "inputs_solvent_SMILES",
        "inputs_base_SMILES",
        "temperature_Celsius",
        "yield_percent"
    ])

    for reaction in dataset.reactions:
        reaction_id = getattr(reaction, "reaction_id", "")

        # Identifiants
        reaction_index_custom = get_identifier_value(reaction.identifiers, 1)
        reaction_type = get_identifier_value(reaction.identifiers, 5)

        # Inputs
        expected_keys = {
            "metal and ligand": "",
            "aryl halide": "",
            "amine": "",
            "solvent": "",
            "base": ""
        }

        for key, val in reaction.inputs.items():
            norm_key = key.strip().lower()
            if norm_key in expected_keys:
                expected_keys[norm_key] = get_smiles_from_components(val.components)

        # Température
        temp_c = ""
        if reaction.conditions and reaction.conditions.temperature and reaction.conditions.temperature.setpoint:
            temp_c = reaction.conditions.temperature.setpoint.value

            yield_type_code = 3
            
        # Rendement
        yield_percent = ""
        for outcome in reaction.outcomes:
            for product in outcome.products:
                if getattr(product, "is_desired_product", False):
                    for meas in product.measurements:
                        if meas.type == yield_type_code:
                            raw_value = getattr(meas.percentage, "value", None)
                            if raw_value is not None:
                                yield_percent = round(1 * raw_value, 2)  # Ex : 0.84 → 84.0

        writer.writerow([
            reaction_id,
            reaction_index_custom,
            reaction_type,
            expected_keys["metal and ligand"],
            expected_keys["aryl halide"],
            expected_keys["amine"],
            expected_keys["solvent"],
            expected_keys["base"],
            temp_c,
            yield_percent
        ])


In [12]:
df = pd.read_csv("test3.csv")
print(df.head())  # affiche les 5 premières lignes


                            reaction_id  reaction_index_CUSTOM  \
0  ord-05542745a3df403aa88ae927045679bd                    900   
1  ord-096c3c09227841d997327413bc9951b7                    441   
2  ord-0986b91ec7734959810ebeac46571d3f                    952   
3  ord-0addbbece6dc4ca2acdd8d8c9131d423                    569   
4  ord-0d9db0bff06f4d53bf85fef23e44393c                    784   

                                       reaction_type  \
0  1.3.1 [N-arylation with Ar-X] Bromo Buchwald-H...   
1  1.3.1 [N-arylation with Ar-X] Bromo Buchwald-H...   
2  1.3.1 [N-arylation with Ar-X] Bromo Buchwald-H...   
3  1.3.1 [N-arylation with Ar-X] Bromo Buchwald-H...   
4  1.3.2 [N-arylation with Ar-X] Chloro Buchwald-...   

   inputs_metal_and_ligand_SMILES  inputs_aryl_halide_SMILES  \
0                             NaN                        NaN   
1                             NaN                        NaN   
2                             NaN                        NaN   
3         